In [2]:
from tensorflow.keras.preprocessing import image
import tensorflow as tf
from tensorflow.keras import applications, layers, models, optimizers, losses, callbacks

In [3]:
IMG_SHAPE = (150,150,3)
# 전이학습 : transfer learning
def create_transfer(base_model):
    x = layers.Input(shape=IMG_SHAPE)
    y = base_model(x)  # y: (4,4,512)
    y = layers.Flatten()(y)
    y = layers.Dense(256, activation='relu')(y)
    y = layers.Dense(1, activation='sigmoid')(y)
    return models.Model(x, y)

In [4]:
# DataGenerator 증강 모드
aug_gen = image.ImageDataGenerator(
    rescale=1.0 / 255.0,  # 스케일 조정
    rotation_range=40,  # 회전 범위
    width_shift_range=0.2,  # 좌우 움직이는 범위
    height_shift_range=0.2,  # 상하 움직이는 범위
    shear_range=0.2,  # 기울기 범위
    zoom_range=0.2,  # 확대 범위
    horizontal_flip=True,  # 좌우반전
    fill_mode="nearest",  # 부족한 부분을 채우는 방법
    brightness_range=[0.8, 1.2])  # 밝기 범위 (1 기준)

# DataGenerator 일반
gen = image.ImageDataGenerator(rescale=1.0 / 255.0)

# Generator 실행
train_gen = aug_gen.flow_from_directory(
    "../../dataset/cats_and_dogs/new_ds/train/",
    target_size=(150, 150),
    batch_size=20,
    class_mode="binary")
valid_gen = gen.flow_from_directory(
    "../../dataset/cats_and_dogs/new_ds/valid/",
    target_size=(150, 150),
    batch_size=20,
    class_mode="binary")

FileNotFoundError: [WinError 3] 지정된 경로를 찾을 수 없습니다: '../../dataset/cats_and_dogs/new_ds/train/'

In [14]:
loss = losses.binary_crossentropy
optimizer = optimizers.RMSprop(learning_rate=2e-5)
metrics = ["acc"]
ckp = callbacks.ModelCheckpoint(filepath="transfer_vgg.h5", save_best_only=True, monitor="val_acc")

In [15]:
# 사전 학습된 모델을 불러온다.
vgg = applications.VGG16(weights='imagenet', #이미지넷 데이터로 사전 학습된 학습 파라미터.
                                include_top=False, # 최상위 층 (flatten이후) 제거. 
                                input_shape= IMG_SHAPE
                                )
vgg.trainable = False
model = create_transfer(vgg)
model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

In [16]:
history = model.fit(train_gen,
                   steps_per_epoch=100,
                   epochs=30,
                   validation_data=valid_gen,
                   validation_steps=50,
                   callbacks=[ckp])

Epoch 1/30


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node model_2/vgg16/block1_conv1/Relu (defined at \AppData\Local\Temp\ipykernel_13760\2384722295.py:1) ]] [Op:__inference_train_function_3485]

Function call stack:
train_function
